In [2]:
#hide
from fastbook import *
from fastai.vision.widgets import *
from fastai.vision.all import *

In [3]:
path= untar_data(URLs.MNIST_SAMPLE)

In [4]:
path.ls()

(#3) [Path('/home/thai_mai/.fastai/data/mnist_sample/train'),Path('/home/thai_mai/.fastai/data/mnist_sample/valid'),Path('/home/thai_mai/.fastai/data/mnist_sample/labels.csv')]

# 1. Preparing the training set

In [5]:
threes= (path/'train'/'3').ls().sorted()
sevens= (path/'train'/'7').ls().sorted()

In [6]:
Image.open(threes[1])

In [12]:
# Converting images to a list of tensors
threes_tens= [tensor(Image.open(o)) for o in threes]
sevens_tens= [tensor(Image.open(o)) for o in sevens]

In [13]:
len(threes), len(threes_tens), type(threes_tens)

(6131, 6131, list)

In [14]:
# Converting the tensor into a stacked
threes_stacked= torch.stack(threes_tens).float()/255
sevens_stacked= torch.stack(sevens_tens).float()/255

In [15]:
threes_stacked.shape

torch.Size([6131, 28, 28])

In [18]:
# Concatenate these two into a dataset
train_x= torch.cat([threes_stacked, sevens_stacked]).view(-1, 28*28)
train_y= tensor([1]* len(threes) + [0]* len(sevens)).unsqueeze(1)

In [21]:
train_x.shape, train_y.shape

(torch.Size([12396, 784]), torch.Size([12396, 1]))

In [22]:
# Create a dataset named ds
ds= list(zip(train_x, train_y))

In [23]:
# Testing an example of ds
x_test, y_test= ds[0]

In [25]:
x_test.shape, y_test

(torch.Size([784]), tensor([1]))

In [26]:
# Converting the dataset ds into DataLoader object
dl= DataLoader(ds, batch_size= 256)

In [27]:
# Testing the first batch
xb_1, yb_1 = first(dl)

In [29]:
xb_1.shape, yb_1.shape

(torch.Size([256, 784]), torch.Size([256, 1]))

# 2. Preparing the valid set

In [30]:
threes_valid= (path/'valid'/'3').ls().sorted()
sevens_valid= (path/'valid'/'7').ls().sorted()

In [32]:
# Converting images to tensors
valid_threes_tens= [tensor(Image.open(o)) for o in threes_valid]
valid_sevens_tens= [tensor(Image.open(o)) for o in sevens_valid]

In [33]:
valid_3_tens= torch.stack(valid_threes_tens).float()/255
valid_7_tens= torch.stack(valid_sevens_tens).float()/255

In [35]:
# Catch two stacked tensor together
valid_x= torch.cat([valid_3_tens, valid_7_tens]).view(-1, 28*28)
valid_y= tensor([1]* len(threes_valid) + [0]* len(sevens_valid)).unsqueeze(1)

In [36]:
# Create a valid dataset
ds_valid= list(zip(valid_x, valid_y))

In [37]:
# Convert to DataLoader
dl_valid= DataLoader(ds_valid, batch_size= 256)

In [38]:
len(dl_valid)

8

# 3. Create a function to calculate the loss

In [42]:
def mnist_loss(model_out, targets):
    # Applying sigmoid to model_out
    probab= model_out.sigmoid() # probab to be 1
    # retunr the loss
    return torch.where(targets==1, 1-probab, probab).mean()

In [44]:
targets= tensor([1,0,1])
probab= tensor([0.9, 0.2, 0.7])

In [45]:
torch.where(targets==1, 1-probab, probab).mean()

tensor(0.2000)

In [47]:
probab= tensor([0.9, 0.2, 0.2])
torch.where(targets==1, 1-probab, probab).mean()

tensor(0.3667)

# 4. Create a function to initialize the parameters

In [48]:
def init_params(size, std= 1.0):
    return (torch.randn(size)*std).requires_grad_()

In [49]:
weights= init_params((28*28,1))
bias= init_params(1)

In [51]:
weights.shape

torch.Size([784, 1])

# 5 . Create a linear function

In [52]:
# Testing the shape of one batch
xb_1.shape

torch.Size([256, 784])

In [53]:
def linear1(xb):
    return xb@weights + bias

# 6. Create a function to calculate the gradient of the loss for one batch

In [54]:
def calc_grad(xb, yb, model):
    # Getting the model_out
    model_out= model(xb)
    # Calculate the loss
    loss= mnist_loss(model_out, yb)
    # Calculate the gradient of the loss
    loss.backward()
    

In [63]:
### An example for the first batch ###
calc_grad(xb_1, yb_1, linear1)

# Getting the gadient
weights.grad.mean(), bias.grad

(tensor(-0.0106), tensor([-0.0711]))

In [64]:
# Setting gradient back to 0
weights.grad.zero_()
bias.grad.zero_()

tensor([0.])

In [65]:
# Calculate again
### An example for the first batch ###
calc_grad(xb_1, yb_1, linear1)

# Getting the gadient
weights.grad.mean(), bias.grad

(tensor(-0.0106), tensor([-0.0711]))

# 7. Create a function to updating the parameter for one epoch

In [66]:
def train_epoch(model, lr, params):
    # loop over the dataset
    for xb, yb in dl:
        # Calculate the gradient for this batch
        calc_grad(xb, yb, model)
        
        for p in params:
            p.data -= lr* p.grad
            
            # Setting p back to 0
            p.grad.zero_()

# 8. Create a function to calculate the accuracy for each batch

In [67]:
def batch_accuracy(xb, yb):
    # Note that xb = model_out
    # Applying sigmoid for xb
    probab= xb.sigmoid()
    correct= (probab>0.5) == yb
    return correct.float().mean()

In [72]:
# Putting all the batches in the dl_valid together
def validate_epoch(model):
    accs= [batch_accuracy(model(xb), yb) for xb, yb in dl_valid]
    return round(torch.stack(accs).mean().item(),4)

# 9. Training for many epochs

In [73]:
# Initialize the parameters
lr= 1.0
weights= init_params((28*28,1))
bias= init_params(1)
params= weights, bias

for i in range(20):
    # Updating the parameters
    train_epoch(linear1, lr, params)
    # Printing the accuracy on the valid set based on the params
    print(validate_epoch(linear1), end= ' ')

0.7113 0.8598 0.9135 0.9345 0.9443 0.9506 0.9551 0.956 0.958 0.9599 0.9609 0.9624 0.9648 0.9648 0.9668 0.9668 0.9672 0.9668 0.9672 0.9672 